In [1]:
# Import Module

from sklearn import utils
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image 
import os 
import cv2
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, Conv3D, MaxPool3D
from keras import losses
from sklearn import datasets
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from tensorflow.keras import layers



In [2]:
tf.__version__


'2.6.4'

# ****Load the Data****

In [3]:
# Function for get data
def getdata(filepath):
  n = 0
  imgs =[]
  for f in os.listdir(filepath):
     if n < 500 :
          img = np.asarray(Image.open(filepath+f))
          img = img / 255.
          imgs.append(img)
          n = n+1

  return np.asarray(imgs)

In [4]:
# Get Data
test_class_0 = getdata("../input/pcam-extended/Pcam_Test_192/Pcam_Test_192/Class0/")
test_class_0.shape


(500, 192, 192, 3)

In [5]:
# Get Data
test_class_1 = getdata("../input/pcam-extended/Pcam_Test_192/Pcam_Test_192/Class1/")
test_class_1.shape

(500, 192, 192, 3)

In [6]:
# Get Data
train_class_0 = getdata("../input/pcam-extended/Pcam_Train/Pcam_Train/Class0/")
train_class_0.shape

(500, 192, 192, 3)

In [7]:
# Get Data
train_class_1 = getdata("../input/pcam-extended/Pcam_Train/Pcam_Train/Class1/")
train_class_1.shape

(500, 192, 192, 3)

In [8]:
# Get Data
valid_class_0 = getdata("../input/pcam-extended/Pcam_Valid/Pcam_Valid/class0/")
valid_class_0.shape

(500, 192, 192, 3)

In [9]:
# Get Data 
valid_class_1 = getdata("../input/pcam-extended/Pcam_Valid/Pcam_Valid/class1/")
valid_class_1.shape

(500, 192, 192, 3)

In [10]:
# Concatinate class 0 with class 1
def concat_classes(class0,class1):
    label_1 = np.ones(500)
    label_0 = np.zeros(500)

    X =np.concatenate((class0, class1))
    y =np.concatenate((label_0, label_1))
    X,y = utils.shuffle(X,y)
    X=np.asarray(X)
    y=np.asarray(y)
    
    
    return X,y
import torchvision.transforms.functional as fn
def normalizee(data):
    for i in data :
        tensor_image = fn.to_tensor(i)
        normalize = fn.normalize(tensor_image)
        data[i] = normalize

    

In [11]:
test_data , test_labels  = concat_classes(test_class_0,test_class_1)


In [12]:
train_data , train_labels  = concat_classes(train_class_0,train_class_1)

In [13]:
valid_data , valid_labels  = concat_classes(valid_class_0,valid_class_1)

In [15]:
valid_data.shape

(1000, 192, 192, 3)

In [15]:

valid_labels = tf.one_hot(valid_labels, depth=2)

train_labels = tf.one_hot(train_labels, depth=2)


test_labels = tf.one_hot(test_labels, depth=2)


2022-12-08 09:37:39.795598: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [16]:
print(train_labels.shape)
print(train_data.shape)
print(valid_labels.shape)
print(valid_data.shape)


(1000, 2)
(1000, 192, 192, 3)
(1000, 2)
(1000, 192, 192, 3)


In [17]:
x = valid_labels[0:700]
valid_labels = valid_labels[700::]

Y = valid_data[0:700]
valid_data = valid_data[700::]


a = test_labels[0:700]
test_labels = test_labels[700::]

b = test_data[0:700]
test_data = test_data[700::]



In [18]:
train_data = np.concatenate((train_data,Y))
train_labels = np.concatenate((train_labels,x))

train_data = np.concatenate((train_data,b))
train_labels = np.concatenate((train_labels,a))




In [19]:
print(train_labels.shape)
print(train_data.shape)

print(valid_labels.shape)
print(valid_data.shape)


print(test_labels.shape)
print(test_data.shape)


(2400, 2)
(2400, 192, 192, 3)
(300, 2)
(300, 192, 192, 3)
(300, 2)
(300, 192, 192, 3)


In [20]:
train_labels[0:10]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [21]:
train_data[0]

array([[[0.96862745, 0.90588235, 1.        ],
        [0.82352941, 0.76078431, 0.8745098 ],
        [0.71372549, 0.65098039, 0.76470588],
        ...,
        [0.87058824, 0.79215686, 0.9372549 ],
        [0.87058824, 0.78823529, 0.94117647],
        [0.59215686, 0.50980392, 0.67843137]],

       [[0.90980392, 0.84313725, 0.97647059],
        [0.95686275, 0.89019608, 1.        ],
        [1.        , 0.95294118, 1.        ],
        ...,
        [0.88235294, 0.8       , 0.95294118],
        [0.76470588, 0.68235294, 0.83529412],
        [0.72156863, 0.63921569, 0.8       ]],

       [[0.88627451, 0.81568627, 0.97254902],
        [0.94117647, 0.87058824, 1.        ],
        [0.79607843, 0.7254902 , 0.8745098 ],
        ...,
        [0.98823529, 0.90588235, 1.        ],
        [0.79607843, 0.71372549, 0.86666667],
        [0.73333333, 0.65490196, 0.8       ]],

       ...,

       [[0.77254902, 0.70588235, 0.87058824],
        [0.78039216, 0.70588235, 0.88235294],
        [0.85490196, 0

# **CNN MODEL**

In [47]:
import tensorflow as tf 
tf.random.set_seed(42)

In [48]:
model = tf.keras.Sequential([
tf.keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu', input_shape = (192,192,3)),
tf.keras.layers.MaxPooling2D(strides = 2),
tf.keras.layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
tf.keras.layers.MaxPooling2D((3, 3),strides = 2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(64,activation = 'relu'),
tf.keras.layers.Dense(2, activation = 'softmax')
])


In [49]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [50]:
history = model.fit(train_data, train_labels, validation_data = (valid_data, valid_labels), epochs = 20 , batch_size = 75)


Epoch 1/20
32/32 [==============================] - 42s 1s/step - loss: 0.9751 - accuracy: 0.6104 - val_loss: 0.5660 - val_accuracy: 0.6767
Epoch 2/20
32/32 [==============================] - 40s 1s/step - loss: 0.5304 - accuracy: 0.7392 - val_loss: 0.4698 - val_accuracy: 0.7767
Epoch 3/20
32/32 [==============================] - 40s 1s/step - loss: 0.4824 - accuracy: 0.7646 - val_loss: 0.5520 - val_accuracy: 0.7033
Epoch 4/20
32/32 [==============================] - 40s 1s/step - loss: 0.4600 - accuracy: 0.7892 - val_loss: 0.4389 - val_accuracy: 0.7833
Epoch 5/20
32/32 [==============================] - 39s 1s/step - loss: 0.4307 - accuracy: 0.8104 - val_loss: 0.4442 - val_accuracy: 0.7667
Epoch 6/20
32/32 [==============================] - 40s 1s/step - loss: 0.3692 - accuracy: 0.8425 - val_loss: 0.4640 - val_accuracy: 0.7633
Epoch 7/20
32/32 [==============================] - 40s 1s/step - loss: 0.3801 - accuracy: 0.8346 - val_loss: 0.4514 - val_accuracy: 0.7900
Epoch 8/20
32/32 [==

In [ ]:
model.summary()

In [51]:
model.evaluate(test_data,test_labels)

10/10 [==============================] - 1s 111ms/step - loss: 0.9431 - accuracy: 0.7733


[0.9431490302085876, 0.7733333110809326]

In [55]:
# save model

from tensorflow.keras.models import model_from_json
modeljson = model.to_json()
with open('model1.json','w') as file:
    file.write(modeljson)
model.save_weights("model1.h5")


# *GRU Model*

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD
from tensorflow.random import set_seed

model1 = tf.keras.Sequential([tf.keras.layers.Reshape((192, 576), input_shape=(192,192,3)),
  tf.keras.layers.GRU(128),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation='softmax')
])

In [35]:
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [36]:
history1 = model1.fit(train_data, train_labels, validation_data = (valid_data, valid_labels), epochs = 40 , batch_size = 75)


Epoch 1/40
32/32 [==============================] - 13s 361ms/step - loss: 0.6956 - accuracy: 0.5533 - val_loss: 0.6820 - val_accuracy: 0.5200
Epoch 2/40
32/32 [==============================] - 11s 337ms/step - loss: 0.6642 - accuracy: 0.5875 - val_loss: 0.6149 - val_accuracy: 0.6467
Epoch 3/40
32/32 [==============================] - 12s 369ms/step - loss: 0.6348 - accuracy: 0.6329 - val_loss: 0.5812 - val_accuracy: 0.6933
Epoch 4/40
32/32 [==============================] - 11s 342ms/step - loss: 0.6106 - accuracy: 0.6583 - val_loss: 0.6017 - val_accuracy: 0.6533
Epoch 5/40
32/32 [==============================] - 11s 340ms/step - loss: 0.6042 - accuracy: 0.6654 - val_loss: 0.5274 - val_accuracy: 0.7300
Epoch 6/40
32/32 [==============================] - 13s 407ms/step - loss: 0.5697 - accuracy: 0.7083 - val_loss: 0.5040 - val_accuracy: 0.7567
Epoch 7/40
32/32 [==============================] - 12s 369ms/step - loss: 0.5606 - accuracy: 0.7167 - val_loss: 0.6025 - val_accuracy: 0.6500

In [37]:
model1.evaluate(test_data,test_labels)

10/10 [==============================] - 1s 89ms/step - loss: 1.0837 - accuracy: 0.7033


[1.0837074518203735, 0.70333331823349]